# Web Scraping using Selenium - Exception Handling - Assignment 3

In [1]:
import numpy as np
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import warnings
warnings.filterwarnings('ignore')
import time
from bs4 import BeautifulSoup
import requests
import urllib.request
import re

1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get("https://www.amazon.in/")
time.sleep(3)

In [4]:
driver.maximize_window()

In [5]:
inp=input("Enter the product to be searched in Amazon India website ")
product=driver.find_element(By.XPATH,'//div[@class="nav-search-field "]/input')
product.send_keys(inp)
search_key = driver.find_element(By.XPATH,'//div[@class="nav-search-submit nav-sprite"]/span')
search_key.click()

Enter the product to be searched in Amazon India website guitar


2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
brandname=[]
productname=[]
price=[]
exchange=[]
expecteddel=[]
availability=[]
url=[]

for pages in range(0,3):
    try:
        producturl=driver.find_elements(By.XPATH,'//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a')
        for i in producturl:
            url.append(i.get_attribute('href'))
    except NoSuchElementException:
        url.append('-')    
try:
    next_button=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    next_button.click()
except NoSuchElementException:
    pass
    
for j in url:
    driver.get(j)
    try:
        brands=driver.find_element(By.XPATH,'//table[@class="a-normal a-spacing-micro"]//tr[1]/td[2]/span')
        brandname.append(brands.text.strip())
    except NoSuchElementException:
            brandname.append('-')
    try:
        products=driver.find_element(By.XPATH,'//div[@id="titleSection"]/h1/span')
        productname.append(products.text.strip())
    except NoSuchElementException:
        productname.append('-')
    try:
        prc=driver.find_element(By.XPATH,'//div[@class="a-section a-spacing-none aok-align-center aok-relative"]/span[3]/span/span[2]')
        price.append(prc.text.strip())
    except NoSuchElementException:
        price.append('-')
    try:
        returns=driver.find_element(By.XPATH,'//div[@id="anonCarousel4"]/ol/li[3]')
        exchange.append(returns.text.strip())
    except NoSuchElementException:
        exchange.append('-')
    try:
        expectdelivery=driver.find_element(By.XPATH,'//div[@id="mir-layout-DELIVERY_BLOCK"]/div/span/span[1]')
        expecteddel.append(expectdelivery.text.strip())
    except NoSuchElementException:
        expecteddel.append('-')
    try:
        stock=driver.find_element(By.XPATH,'//div[@id="availability"]')
        availability.append(stock.text.strip())
    except NoSuchElementException:
        availability.append('-')

df=pd.DataFrame({'Brand Name':brandname,'Name of the Product':productname,'Price':price,'Return/Exchange':exchange,'Expected Delivery':expecteddel,'Availability':availability,'Product URL':url})
df

,Brand Name,Name of the Product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Kadence,"Kadence rosewood Guitar Frontier Series, Elect...","5,399",-,"Saturday, 8 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Kadence,Kadence Frontier guitar with Online Guitar lea...,"4,899",-,"Friday, 7 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Henrix,Henrix 38C PRO 38 Inch Cutaway Acoustic Guitar...,"3,599",-,"Friday, 7 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,"9,999",-,"Friday, 7 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,JUAREZ,"Juârez Acoustic Guitar Kit, 38 Inch Cutaway, 0...","2,419",-,"Monday, 10 June",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
175,blueberry,"BLUEBERRY D38S, Acoustic Guitar Kit Inbuilt Tr...","2,759",-,"Monday, 10 June",In stock,https://www.amazon.in/Blueberry-Acoustic-Guita...
176,Gum-Tea,"Acoustic Guitar, 39 Inch Cutaway, 039C with Ba...","4,997",-,"Thursday, 13 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
177,Black,SG Musical - Guitar Floor Stand Foldable and A...,549,-,"Tuesday, 11 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
178,Black,SG Musical - Metal Guitar Stand - Universal Gu...,599,-,"Tuesday, 11 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [7]:
df.to_csv(r'Downloads\amazon.csv')

In [8]:
df5=pd.read_csv(r"Downloads\amazon.csv")
df5.head()

,Unnamed: 0,Brand Name,Name of the Product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,0,Kadence,"Kadence rosewood Guitar Frontier Series, Elect...","5,399",-,"Saturday, 8 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,1,Kadence,Kadence Frontier guitar with Online Guitar lea...,"4,899",-,"Friday, 7 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,2,Henrix,Henrix 38C PRO 38 Inch Cutaway Acoustic Guitar...,"3,599",-,"Friday, 7 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,3,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,"9,999",-,"Friday, 7 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,4,JUAREZ,"Juârez Acoustic Guitar Kit, 38 Inch Cutaway, 0...","2,419",-,"Monday, 10 June",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...


3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [119]:
driver=webdriver.Chrome()

In [120]:
driver.get("https://images.google.com/")

In [121]:
driver.maximize_window()

In [122]:
image_list=["fruits","cars and Machine Learning","Guitar","Cakes"]

In [123]:
fruitss=[]
carandml=[]
guitar=[]
cake=[]

for i in image_list:
    search=driver.find_element(By.XPATH,'//div[@class="a4bIc"]/textarea')
    search.send_keys(i)
    search_click=driver.find_element(By.XPATH,'//div[@class="zgAlFc"]/span')
    search_click.click()
    time.sleep(10)
    driver.execute_script("window.scrollBy(0,5000)","")
    try:
        if "fruits" in i:
            fruits=driver.find_elements(By.XPATH,'//img[@class="YQ4gaf"]')
            for j in fruits:
                source=j.get_attribute('src')
                if source is not None:
                    if(source[0:4]=="http"):
                        fruitss.append(source)
                        fruitss=fruitss[0:10]
            #print(fruitss)
            clear_search=driver.find_element(By.XPATH,'//div[@class="M2vV3 vOY7J"]/span')
            clear_search.click()
    except NoSuchElementException:
        pass
    try:
        if "cars and Machine Learning" in i:
            cars=driver.find_elements(By.XPATH,'//img[@class="YQ4gaf"]')
            for j in cars:
                source=j.get_attribute('src')
                if source is not None:
                    if(source[0:4]=="http"):
                        carandml.append(source)
                        carandml=carandml[0:10]
            #print(carandml)
            clear_search=driver.find_element(By.XPATH,'//div[@class="M2vV3 vOY7J"]/span')
            clear_search.click()
    except NoSuchElementException:
        pass
    try:
        if "Guitar" in i:
            guitars=driver.find_elements(By.XPATH,'//img[@class="YQ4gaf"]')
            for j in guitars:
                source=j.get_attribute('src')
                if source is not None:
                    if(source[0:4]=="http"):
                        guitar.append(source)
                        guitar=guitar[0:10]
            #print(carandml)
            clear_search=driver.find_element(By.XPATH,'//div[@class="M2vV3 vOY7J"]/span')
            clear_search.click()
    except NoSuchElementException:
        pass
    try:
        if "Cakes" in i:
            cakes=driver.find_elements(By.XPATH,'//img[@class="YQ4gaf"]')
            for j in cakes:
                source=j.get_attribute('src')
                if source is not None:
                    if(source[0:4]=="http"):
                        cake.append(source)
                        cake=cake[0:10]
            #print(carandml)
            clear_search=driver.find_element(By.XPATH,'//div[@class="M2vV3 vOY7J"]/span')
            clear_search.click()
    except NoSuchElementException:
        pass

for k in range(len(fruitss)):
    if k >= 10:
        break
    print("Downloading {0} of {1} images for fruits" .format(k, 10))
    response= requests.get(fruitss[k])
    file = open(r"C:\Users\DELL\OneDrive\Documents"+str(k)+".jpg", "wb")
    file.write(response.content)  
for l in range(len(carandml)):
    if l >= 10:
        break
    print("Downloading {0} of {1} images for cars and machine learning" .format(l, 10))
    response= requests.get(carandml[l])
    file = open(r"C:\Users\DELL\OneDrive\Documents"+str(l)+".jpg", "wb")
    file.write(response.content)  
for m in range(len(guitar)):
    if m >= 10:
        break
    print("Downloading {0} of {1} images for guitar" .format(m, 10))
    response= requests.get(guitar[m])
    file = open(r"C:\Users\DELL\OneDrive\Documents"+str(m)+".jpg", "wb")
    file.write(response.content)  
for n in range(len(cake)):
    if n >= 10:
        break
    print("Downloading {0} of {1} images for cake" .format(n, 10))
    response= requests.get(cake[n])
    file = open(r"C:\Users\DELL\OneDrive\Documents"+str(n)+".jpg", "wb")
    file.write(response.content)  

In [201]:
df.to_csv(r"Downloads\image.csv")

In [190]:
print(range(len(df)))

range(0, 10)


4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [115]:
driver=webdriver.Chrome()

In [116]:
driver.get("https://www.flipkart.com/")
time.sleep(3)

In [117]:
driver.maximize_window()

In [118]:
search=driver.find_element(By.XPATH,'//div[@class="_2SmNnR"]/input')
search.send_keys("smartphone")
search.send_keys(Keys.ENTER)

In [112]:
phone=[]
brandname=[]
smartphone=[]
colour=[]
RAM=[]
storage=[]
primarycam=[]
secondarycam=[]
display=[]
battery=[]
price=[]

try:
    phones=driver.find_elements(By.XPATH,'//a[@class="CGtC98"]')
    for i in phones:
        phone.append(i.get_attribute('href'))
except NoSuchElementException:
    phone.append('-')

for j in phone:
    driver.get(j)
    try:
        brands=driver.find_element(By.XPATH,'//span[@class="VU-ZEz"]')
        brandname.append(brands.text.strip().split(' ')[0])
    except NoSuchElementException:
        brandname.append('-')
    try:
        smarts=driver.find_element(By.XPATH,'//span[@class="VU-ZEz"]')
        smartphone.append(smarts.text.strip().split('(')[0])
    except NoSuchElementException:
        smartphone.append('-')
    try:
        confg=driver.find_element(By.XPATH,'//div[@class="GNDEQ-"][1]//table[1]//tr[4]/td[2]/ul/li[1]')
        colour.append(confg.text.strip())
    except NoSuchElementException:
        colour.append('-')
    button=driver.find_element(By.XPATH,'//button[@class="QqFHMw _4FgsLt"]')
    button.click()
    try:
        rams=driver.find_element(By.XPATH,'//div[@class="GNDEQ-"][4]//table[1]//tr[2]/td[2]/ul/li[1]')
        RAM.append(rams.text.strip())
    except NoSuchElementException:
        RAM.append('-')
    try:
        roms=driver.find_element(By.XPATH,'//div[@class="GNDEQ-"][4]//table[1]//tr[1]/td[2]/ul/li[1]')
        storage.append(roms.text.strip())
    except NoSuchElementException:
        storage.append('-')
    try:
        primary=driver.find_element(By.XPATH,'//div[@class="U+9u4y"]/div[2]/ul/li[3]')
        primarycam.append(primary.text.strip().split('|')[0])
    except NoSuchElementException:
        primarycam.append('-')
    try:
        secondary=driver.find_element(By.XPATH,'//div[@class="GNDEQ-"][5]//table[1]//tr[5]/td[2]/ul/li[1]')
        secondarycam.append(secondary.text.strip())
    except NoSuchElementException:
        secondarycam.append('-')
    try:
        size=driver.find_element(By.XPATH,'//div[@class="GNDEQ-"][2]//table[1]//tr[1]/td[2]/ul/li[1]')
        display.append(size.text.strip())
    except NoSuchElementException:
        display.append('-')
    try:
        power=driver.find_element(By.XPATH,'//div[@class="U+9u4y"]/div[2]/ul/li[4]')
        battery.append(power.text.strip())
    except NoSuchElementException:
        battery.append('-')
    try:
        prc=driver.find_element(By.XPATH,'//div[@class="UOCQB1"]/div/div[1]')
        price.append(prc.text.strip())
    except NoSuchElementException:
        price.append('-')

df2=pd.DataFrame({"Brand Name":brandname,"Smartphone name":smartphone,"Colour":colour,"RAM":RAM,"Storage(ROM)":storage,"Primary Camera":primarycam,"Secondary Camera":secondarycam,"Display Size":display,"Battery Capacity":battery,"Price":price,"Product URL":phone})
df2

,Brand Name,Smartphone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,Product URL
0,POCO,POCO F6 5G,Black,12 GB,256 GB,50MP (OIS) + 8MP,20MP Front Camera,16.94 cm (6.67 inch),5000 mAh Battery,"₹31,999",https://www.flipkart.com/poco-f6-5g-black-256-...
1,POCO,POCO F6 5G,Titanium,8 GB,256 GB,50MP (OIS) + 8MP,20MP Front Camera,16.94 cm (6.67 inch),5000 mAh Battery,"₹29,999",https://www.flipkart.com/poco-f6-5g-titanium-2...
2,Motorola,Motorola G34 5G,Ocean Green,8 GB,128 GB,50MP + 2MP,16MP Front Camera,16.51 cm (6.5 inch),5000 mAh Battery,"₹11,999",https://www.flipkart.com/motorola-g34-5g-ocean...
3,Motorola,Motorola G34 5G,Charcoal Black,8 GB,128 GB,50MP + 2MP,16MP Front Camera,16.51 cm (6.5 inch),5000 mAh Battery,"₹11,999",https://www.flipkart.com/motorola-g34-5g-charc...
4,POCO,POCO M6 Pro 5G,Power Black,6 GB,128 GB,50MP + 2MP,8MP Front Camera,17.25 cm (6.79 inch),5000 mAh Battery,"₹9,999",https://www.flipkart.com/poco-m6-pro-5g-power-...
5,POCO,POCO M6 Pro 5G,Forest Green,6 GB,128 GB,50MP + 2MP,8MP Front Camera,17.25 cm (6.79 inch),5000 mAh Battery,"₹9,999",https://www.flipkart.com/poco-m6-pro-5g-forest...
6,POCO,POCO F6 5G,Black,8 GB,256 GB,50MP (OIS) + 8MP,20MP Front Camera,16.94 cm (6.67 inch),5000 mAh Battery,"₹29,999",https://www.flipkart.com/poco-f6-5g-black-256-...
7,POCO,POCO F6 5G,Titanium,12 GB,512 GB,50MP (OIS) + 8MP,20MP Front Camera,16.94 cm (6.67 inch),5000 mAh Battery,"₹33,999",https://www.flipkart.com/poco-f6-5g-titanium-5...
8,Motorola,Motorola G34 5G,Ice Blue,8 GB,128 GB,50MP + 2MP,16MP Front Camera,16.51 cm (6.5 inch),5000 mAh Battery,"₹11,999",https://www.flipkart.com/motorola-g34-5g-ice-b...
9,Motorola,Motorola g64 5G,Ice Lilac,8 GB,128 GB,50MP (OIS) + 8MP,16MP Front Camera,16.51 cm (6.5 inch),6000 mAh Battery,"₹14,999",https://www.flipkart.com/motorola-g64-5g-ice-l...


In [114]:
df2.to_csv(r"Downloads\smartphone.csv")

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [63]:
driver=webdriver.Chrome()

In [64]:
driver.get("https://www.google.com/maps")

In [65]:
driver.maximize_window()

In [66]:
try:
    adclose=driver.find_element(By.XPATH,'//span[@class="AmPKde google-symbols"]')
    adclose.click()
except NoSuchElementException:
    pass

In [67]:
inp=input("Enter the city to be searched in google maps ")
search=driver.find_element(By.XPATH,'//form[@class="NhWQq"]/input')
search.send_keys(inp)
search.send_keys(Keys.ENTER)
time.sleep(7)
url=driver.current_url
print("URL:",url)
a=re.findall(r'@(.*),',url)
print("Latitude:",a[0].split(',')[0])
print("Longitude:",a[0].split(',')[1])

Enter the city to be searched in google maps bombay
URL: https://www.google.com/maps/place/Mumbai,+Maharashtra/@19.0826068,72.5515025,10z/data=!3m1!4b1!4m6!3m5!1s0x3be7c6306644edc1:0x5da4ed8f8d648c69!8m2!3d19.0759837!4d72.8776559!16zL20vMDR2bXA?entry=ttu
Latitude: 19.0826068
Longitude: 72.5515025


6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [66]:
driver=webdriver.Chrome()

In [67]:
driver.get("https://www.digit.in/top-products/best-gaming-laptops-40.html")

In [68]:
driver.maximize_window()

In [69]:
url=[]
brand=[]
os=[]
model=[]
display=[]
connectivity=[]
ram=[]
processor=[]
processmodel=[]
drive=[]
drivetype=[]
batterytype=[]



laptop=driver.find_elements(By.XPATH,'//h3[@class="font130 mt0 mb10 mobilesblockdisplay "]/a')
for i in laptop:
    url.append(i.get_attribute('href'))
for j in url:
    driver.get(j)
    brands=driver.find_element(By.XPATH,'//h1[@class="floatleft tabletblockdisplay pr20 "]')
    brand.append(brands.text.split(' ')[0])
    names=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[2]/td/table//tr[3]/td')
    model.append(names.text.strip())
    opersys=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[2]/td/table//tr[1]/td')
    os.append(opersys.text.strip())
    displays=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[4]//td/table//tr[1]/td')
    display.append(displays.text.strip())
    connect=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[6]//td/table//tr[1]/td')
    connectivity.append(connect.text.strip())
    rams=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[8]//td/table//tr[1]/td')
    ram.append(rams.text.strip())
    process=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[10]//td/table//tr[1]/td')
    processor.append(process.text.strip())
    procmodel=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[10]//table//tr[2]/td')
    processmodel.append(procmodel.text.strip())
    drives=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[12]//table//tr[1]/td')
    drive.append(drives.text.strip())
    drivestype=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[12]//table//tr[2]/td')
    drivetype.append(drivestype.text.strip())
    battery=driver.find_element(By.XPATH,'//table[@class="woocommerce-group-attributes-public"]//tr[14]//table//tr[1]/td')
    batterytype.append(battery.text.strip())

df=pd.DataFrame({"Brand":brand,"Model":model,"Operating system":os,"Display":display,"Connectivity":connectivity,"RAM":ram,"Processor":processor,"Processor Model":processmodel,"Storage":drive,"Storage Type":drivetype,"Battery":batterytype,"Product URL":url})
df

,Brand,Model,Operating system,Display,Connectivity,RAM,Processor,Processor Model,Storage,Storage Type,Battery,Product URL
0,MSI,Titan GT77HX 13VI-092IN,Windows 11 Home,17.3,"Bluetooth, Wi-Fi",64 GB,5.6 GHz,24,4 TB,SSD,"4 Cell, 99 WHr",https://www.digit.in/laptops/msi-titan-gt77-hx...
1,ASUS,ROG Strix SCAR 16,Windows 11 Home,16,"Bluetooth, Wi-Fi",32 GB,5.8 GHz,36 MB,2 TB,SSD,"4 Cell, 90 WHr",https://www.digit.in/laptops/asus-rog-strix-sc...
2,MSI,Raider GE78HX 13VI,Windows 11 Home,17,"Bluetooth, Wi-Fi",64 GB,NVIDIA GeForce RTX 4090,13th Gen Intel Core i9-13980HX,2 TB,SSD,4 Cell 98 WHr,https://www.digit.in/laptops/msi-raider-ge78hx...
3,ASUS,ROG Strix SCAR 18,Windows 11 Home,18,"Bluetooth, Wi-Fi",16 GB,5.6 GHz,36 MB,2 TB,SSD,"4 Cell, 90 WHr",https://www.digit.in/laptops/asus-rog-strix-sc...
4,Lenovo,Legion Pro 7i,Windows 11 Home,16,"Bluetooth, Wi-Fi",32 GB,5.40 GHz,NVIDIA GeForce RTX 4090,1 TB,SSD,"4 Cell, 99.9 WHr",https://www.digit.in/laptops/lenovo-legion-pro...
5,OMEN,17-ck2004TX,Windows 11 Home,17.3,"Bluetooth, Wi-Fi",32 GB,5.4 GHz,36 MB,1 TB,SSD,"6 Cell, 83 WHr",https://www.digit.in/laptops/omen-by-hp-17-ck2...
6,Dell,Alienware M18 R1,Windows 11 Home,18,"Bluetooth, Wi-Fi",32 GB,5.4 GHz,36 MB,1 TB,SSD,"6 Cell, 97 WHr",https://www.digit.in/laptops/dell-alienware-m1...
7,Dell,Alienware X16 R1,Windows 11 Home,16,"Bluetooth, Wi-Fi",32 GB,5.4 GHz,24 MB,1 TB,SSD,"6 Cell, 90 WHr",https://www.digit.in/laptops/dell-alienware-x1...
8,MSI,Raider GE67HX 12UHS,Windows 11 Home,15.6,"Bluetooth, Wi-Fi",32 GB,5 GHz,NVIDIA GeForce RTX 3080 Ti,1 TB,SSD,"4 Cell, 99 WHr",https://www.digit.in/laptops/msi-raider-ge67hx...
9,Acer,Predator Helios 16,Windows 11 Home,16,"Bluetooth, Wi-Fi",32 GB,NVIDIA GeForce RTX 4080,13th Gen Intel Core i9-13900HX,1 TB,SSD,4 Cell 90 WHr,https://www.digit.in/laptops/acer-predator-hel...


7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [19]:
driver=webdriver.Chrome()

In [20]:
driver.get("https://www.forbes.com/billionaires/")
time.sleep(3)

In [21]:
driver.maximize_window()

In [9]:
close_row=driver.find_element(By.XPATH,'//div[@class="Table_tableRow__lF_cY Table_expanded__0lLQ6"]/div[1]')
close_row.click()
#driver.execute_script("window.scrollBy(0,100000)","")

In [22]:
rank=[]
heads=[]
name=[]
net=[]
profile=[]
age=[]
citizen=[]
source=[]
industry=[]

for i in range(0,13):
    header=driver.find_elements(By.XPATH,'//div[@class="Table_tableRow__lF_cY"]')
    for j in header:
        rank.append(j.text.strip().split("\n")[0])
        name.append(j.text.strip().split("\n")[1])
        net.append(j.text.strip().split("\n")[2])
        age.append(j.text.strip().split("\n")[3])
        citizen.append(j.text.strip().split("\n")[4])
        source.append(j.text.strip().split("\n")[5])
        industry.append(j.text.strip().split("\n")[6])
    next_button=driver.find_element(By.XPATH,'//button[@class="t7aCs7DQ _0BYTGi5u TNAO5JyH bMlalnxk _5nHoWK7c"][2]')
    next_button.click()
df=pd.DataFrame({"Rank":rank,"Name":name,"Net worth":net,"Age":age,"Citizenship":citizen,"Source":source,"Industry":industry})
df

,Rank,Name,Net worth,Age,Citizenship,Source,Industry
0,2.,Elon Musk,$195 B,52,United States,"Tesla, SpaceX",Automotive
1,3.,Jeff Bezos,$194 B,60,United States,Amazon,Technology
2,4.,Mark Zuckerberg,$177 B,39,United States,Facebook,Technology
3,5.,Larry Ellison,$141 B,79,United States,Oracle,Technology
4,6.,Warren Buffett,$133 B,93,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
2582,2545.,Huang Jiangji,$1.1 B,N/A,Hong Kong,smartphones,Technology
2583,2545.,Subbamma Jasti,$1.1 B,91,India,Pharmaceuticals,Healthcare
2584,2545.,Morris Kahn,$1.1 B,94,Israel,Software,Technology
2585,2545.,Jonas Kamprad,$1.1 B,58,Sweden,IKEA,Finance & Investments


8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video. 

In [206]:
driver=webdriver.Chrome()

In [207]:
driver.get("https://www.youtube.com/watch?v=Z5DcTdVqqTI&t=5s")

In [208]:
driver.maximize_window()

In [209]:
driver.execute_script("window.scrollBy(0,500)","")

In [210]:
comments_header=[]
comments=[]
upvotes=[]
times=[]

driver.execute_script("window.scrollBy(0,100000)","")
time.sleep(30)
timer=driver.find_elements(By.XPATH,'//span[@id="published-time-text"]/a')
for i in timer:
    times.append(i.text)
comment=driver.find_elements(By.XPATH,'//yt-attributed-string[@class="style-scope ytd-comment-view-model"]/span')
for j in comment:
    comments.append(j.text)
driver.execute_script("window.scrollBy(0,100000)","")
time.sleep(30)
upvote=driver.find_elements(By.XPATH,'//span[@id="vote-count-middle"]')
for k in upvote:
    upvotes.append(k.text)
print(comments)
print(times)
print(upvotes)

['I literally watch these to fall asleep', 'Even though you cant do them in reality, you have to admit these videos are quite soothing at night', 'Even on a school night I never can fall asleep so I always watch these to help me fall asleep', 'Who always watches this ,but never tries it', 'Literally no one:\n\nMe: Is always overthinking what could go wrong with these life hacks-', 'Thank you I needed to know how to put a straw in my potato', 'Brother: where did all my shirts go\nMe: 5 minute craft took them all', '2:40 Ah yes… I always hate it when my straw won’t go into my potato.', 'Nobody: \n\n\nLife Hack YouTubers: hOw tO pUt a sTrAw iN yOuR pHoNe', "I now know how to stick a straw into a raw potato. That's been worrying me for years, I've had therapy, lost countless hours sleep...thank you ", 'Oh i hate it when my straw  won’t go in my potato ', 'I’ve never known someone to eat one spoonful of yogurt then feel the need to seal it back up ', 'Omg finally!!! I’ve waited my whole lif

9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [87]:
driver=webdriver.Chrome()

In [88]:
driver.get("https://www.hostelworld.com/")
time.sleep(3)

In [89]:
driver.maximize_window()

In [90]:
search=driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/main/header/div/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/label/input')
search.send_keys("London")
time.sleep(3)

In [91]:
select=driver.find_element(By.XPATH,'//div[@class="select-list-container destination-search-results"]/ul/li[2]/button')
select.click()

In [92]:
search_click=driver.find_element(By.XPATH,'//button[@class="large-button btn-content"]/div')
search_click.click()
time.sleep(7)

In [93]:
hostel=[]
info=[]
distance=[]
ratings=[]
totreviews=[]
overreviews=[]
private=[]
dorms=[]
facilities=[]
description=[]


data=driver.find_elements(By.XPATH,'//div[@class="property-info-container"]')
for i in data:
    name=i.find_element(By.CLASS_NAME,'property-name')
    hostel.append(name.text)
    rating=i.find_element(By.CLASS_NAME,'number')
    ratings.append(rating.text)
dorm=driver.find_elements(By.XPATH,'//div[@class="property-accommodations"]/div/div[2]')
for n in dorm:
    dorms.append(n.text.replace("\n",' ').split(' '))
privates=driver.find_elements(By.XPATH,'//div[@class="property-accommodations"]/div/div[1]')
for o in privates:
    private.append(o.text.replace("\n",' ').split(' '))
link=driver.find_elements(By.XPATH,'//div[@class="property-card"]/a/a')
for k in link:
    info.append(k.get_attribute('href'))
for l in info:
    driver.get(l)
    total=driver.find_element(By.XPATH,'//div[@class="reviews-and-ratings"]/div[1]/div/div[2]/div[2]')
    totreviews.append(total.text)
    over=driver.find_element(By.XPATH,'//div[@class="reviews-and-ratings"]/div[1]/div/div[2]/div[1]/span')
    overreviews.append(over.text)
    facility=driver.find_element(By.XPATH,'//div[@class="facilities-container"]/ul')
    facilities.append(facility.text.strip().replace("\n",','))
    properties=driver.find_element(By.XPATH,'//div[@class="initial-property-info"]/div[2]')
    description.append(properties.text.strip().replace("\n",' '))
df=pd.DataFrame({"hostel name":hostel,"ratings":ratings,"total reviews":totreviews,"overall reviews":overreviews,"facilities":facilities,"property description":description,"URL":info})
df

,hostel name,ratings,total reviews,overall reviews,facilities,property description,URL
0,Palmers Lodge - Swiss Cottage,8.7,16199 Total Reviews,Fabulous,"Free,Linen Included Free WiFi Free Internet Ac...",Hostelworld says Palmers Lodge Swiss Cottage i...,https://www.hostelworld.com/pwa/hosteldetails....
1,St Christopher's Village,7.5,13078 Total Reviews,Very Good,"Free,Linen Included Free City Maps Free WiFi F...",Hostelworld says Hostelworld visitors have sai...,https://www.hostelworld.com/pwa/hosteldetails....
2,Wombat's City Hostel London,9.3,16481 Total Reviews,Superb,"Free,Linen Included Free City Maps Free WiFi F...","Hostelworld says Wombat's City Hostel London, ...",https://www.hostelworld.com/pwa/hosteldetails....
3,Palmers Lodge - Swiss Cottage,8.7,16199 Total Reviews,Fabulous,"Free,Linen Included Free WiFi Free Internet Ac...",Hostelworld says Palmers Lodge Swiss Cottage i...,https://www.hostelworld.com/pwa/hosteldetails....
4,Onefam Waterloo,9.5,343 Total Reviews,Superb,"Free,Linen Included Free WiFi Free Internet Ac...",Property Description Come join us at our newes...,https://www.hostelworld.com/pwa/hosteldetails....
5,NX London Hostel,8.8,2367 Total Reviews,Fabulous,"Free,Free Breakfast Linen Included Towels Incl...",Property Description Welcome to NX London Host...,https://www.hostelworld.com/pwa/hosteldetails....
6,Urbany Hostel London,9.6,1277 Total Reviews,Superb,"Free,Linen Included Free WiFi,General,Security...",Property Description Welcome to Urbany Hostel ...,https://www.hostelworld.com/pwa/hosteldetails....
7,Onefam Notting Hill,9.7,2819 Total Reviews,Superb,"Free,Linen Included Free WiFi Free Internet Ac...",Property Description The perfect place for sol...,https://www.hostelworld.com/pwa/hosteldetails....
8,St Christopher's Village,7.5,13078 Total Reviews,Very Good,"Free,Linen Included Free City Maps Free WiFi F...",Hostelworld says Hostelworld visitors have sai...,https://www.hostelworld.com/pwa/hosteldetails....
9,Generator London,7.2,8261 Total Reviews,Very Good,"Free,Linen Included Free City Maps Free WiFi F...",Hostelworld says Hostelworld guests have said ...,https://www.hostelworld.com/pwa/hosteldetails....


In [ ]:
hostel=[]
info=[]
distance=[]
ratings=[]
totreviews=[]
overreviews=[]
privates=[]
dorms=[]
facilities=[]
description=[]


data=driver.find_elements(By.XPATH,'//div[@class="property-info-container"]')
for i in data:
    name=i.find_element(By.CLASS_NAME,'property-name')
    hostel.append(name.text)
    rating=i.find_element(By.CLASS_NAME,'number')
    ratings.append(rating.text)
try:
    dist=driver.find_elements(By.XPATH,'//div[@class="property-distance"]/span[2]')
    for j in dist:
        distance.append(j.text)
except NoSuchElementException:
    distance.append('-')

private=driver.find_elements(By.XPATH,'//div[@class="property-accommodations"]/div/div[2]')
for n in private:
    privates.append(n.text.split("\n")[1])

link=driver.find_elements(By.XPATH,'//div[@class="property-card"]/a/a')
for k in link:
    info.append(k.get_attribute('href'))
for l in info:
    driver.get(l)
    total=driver.find_element(By.XPATH,'//div[@class="rating-container collapsed"]/div/div/div[2]/span')
    totreviews.append(total.text)
    over=driver.find_element(By.XPATH,'//div[@class="rating column large"]/div/span[2]')
    overreviews.append(over.text)
    #facility=driver.find_element(By.XPATH,'//div[@class="about-facilities"]/button/div')
    #facility.click()
    fac=driver.find_elements(By.XPATH,'//div[@class="facilities-category-container"]')
    for m in fac:
        facilities.append(m.text.strip().replace("\n",','))
print(len(hostel),len(ratings),len(distance),len(info),len(totreviews),len(overreviews),len(facilities),len(privates))
print(privates)

In [ ]:
#//div[@class="inline-wrapper hp-search-form-desktop"]/div/div/div[1]/div/div[2]/label/input

In [ ]:
#//div[@class="initial-property-info"]/div[2]/div[6]/div/div[2]